## Confirmed Cases by Province/State (for Leaflet map)

In [1]:
import pandas as pd
import os
import json
import requests
from pprint import pprint

In [2]:
# Global Cases (minus U.S. total)
confirmed_df = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
us = confirmed_df.loc[confirmed_df['Country/Region']=='US']
confirmed_df.drop(us.index[0], axis=0, inplace=True)
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,444,484,521,555,607,665,714,784,840,906
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,400,409,416,433,446,467,475,494,518,539
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1572,1666,1761,1825,1914,1983,2070,2160,2268,2418
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,564,583,601,601,638,646,659,673,673,696
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,19,19,19,19,19,19,19,19,19,19


In [3]:
# U.S. Cases by State
confirmed_us_df = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_us_df = confirmed_us_df.iloc[:,6:]
confirmed_us_df.drop(['Combined_Key'], axis=1, inplace=True)
confirmed_us_df.rename(columns={'Province_State': 'Province/State',
                               'Country_Region': 'Country/Region',
                               'Long_': 'Long'}, inplace=True)
print(len(confirmed_us_df))
confirmed_us_df.head()

3255


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20
0,American Samoa,US,-14.2710,-170.1320,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,13.4443,144.7937,0,0,0,0,0,0,...,121,128,130,133,133,133,133,135,135,136
2,Northern Mariana Islands,US,15.0979,145.6739,0,0,0,0,0,0,...,11,11,11,11,11,11,11,13,13,13
3,Puerto Rico,US,18.2208,-66.5901,0,0,0,0,0,0,...,620,683,725,788,897,903,923,974,1043,1068
4,Virgin Islands,US,18.3358,-64.8963,0,0,0,0,0,0,...,45,45,50,51,51,51,51,51,51,51


In [4]:
# Drop rows with zero cases
last_column = confirmed_us_df[confirmed_us_df.columns[-1]]
rows = confirmed_us_df.loc[last_column == 0].index
confirmed_us_df.drop(rows, inplace=True)

# Drop cruise ship with no location
cruises = confirmed_us_df.loc[confirmed_us_df['Province/State'].str.contains('Grand Princess')].index
confirmed_us_df.drop(cruises, inplace=True)

print(len(confirmed_us_df))
confirmed_us_df.tail()

2727


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20
3243,Tennessee,US,0.0,0.0,0,0,0,0,0,0,...,31,38,22,19,12,46,136,136,197,209
3246,Vermont,US,0.0,0.0,0,0,0,0,0,0,...,33,8,59,11,7,8,39,15,15,10
3248,Washington,US,0.0,0.0,0,0,0,0,0,0,...,915,838,908,848,732,619,484,383,272,334
3253,Michigan,US,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,370,429,472,472,514
3254,Michigan,US,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,21,23,36,36,44


In [15]:
# Use previous lat, long so initial GMaps API doesn't have to run again.
previous = pd.read_csv('confirmed.csv')

for index, row in confirmed_us_df.iterrows():
    state = row['Province/State']
    if ((row['Lat']==0) and (row['Long']==0) and (row['Country/Region']=='US')):
        state = row['Province/State']
        latlong = previous.loc[previous['Province/State']==state]
        lat = latlong.iloc[-1,2]
        long = latlong.iloc[-1,3]
        confirmed_us_df.loc[index, 'Lat'] = lat
        confirmed_us_df.loc[index, 'Long'] = long

In [16]:
# def lat_long(state):

#     url = "https://google-maps-geocoding.p.rapidapi.com/geocode/json"

#     querystring = {"language":"en","address":state}

#     headers = {
#         'x-rapidapi-host': "google-maps-geocoding.p.rapidapi.com",
#         'x-rapidapi-key': GMAPS_API_KEY
#         }

#     response = requests.get(url, headers=headers, params=querystring).json()

#     location = response['results'][0]['geometry']['location']
#     lat = location['lat']
#     long = location['lng']

#     return lat, long

In [17]:
# for index, row in confirmed_us_df.iterrows():
#     state = row['Province/State']
#     if ((row['Lat']==0) and (row['Long']==0) and (row['Country/Region']=='US')):
# #         try:
#         lat, long = lat_long(state)
#         confirmed_us_df.loc[index, 'Lat'] = lat
#         confirmed_us_df.loc[index, 'Long'] = long
# #         except (IndexError, KeyError): 
# #             print(index, state)
# #             confirmed_us_df.drop(row, inplace=True)
# #             pass

# print(len(confirmed_us_df))

In [18]:
confirmed_us_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20
1,Guam,US,13.444300,144.793700,0,0,0,0,0,0,...,121,128,130,133,133,133,133,135,135,136
2,Northern Mariana Islands,US,15.097900,145.673900,0,0,0,0,0,0,...,11,11,11,11,11,11,11,13,13,13
3,Puerto Rico,US,18.220800,-66.590100,0,0,0,0,0,0,...,620,683,725,788,897,903,923,974,1043,1068
4,Virgin Islands,US,18.335800,-64.896300,0,0,0,0,0,0,...,45,45,50,51,51,51,51,51,51,51
5,Alabama,US,32.539527,-86.644082,0,0,0,0,0,0,...,12,15,17,19,19,19,23,24,26,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,Tennessee,US,35.517491,-86.580447,0,0,0,0,0,0,...,31,38,22,19,12,46,136,136,197,209
3246,Vermont,US,44.558803,-72.577841,0,0,0,0,0,0,...,33,8,59,11,7,8,39,15,15,10
3248,Washington,US,47.751074,-120.740139,0,0,0,0,0,0,...,915,838,908,848,732,619,484,383,272,334
3253,Michigan,US,44.314844,-85.602364,0,0,0,0,0,0,...,0,0,0,0,0,370,429,472,472,514


In [19]:
# Combine both Global and U.S.
combined_df = pd.concat([confirmed_df, confirmed_us_df], sort=False)
combined_df.drop(combined_df.columns[4:-1], axis=1, inplace=True)
combined_df.head()

,Province/State,Country/Region,Lat,Long,4/17/20
0,NaN,Afghanistan,33.0000,65.0000,906
1,NaN,Albania,41.1533,20.1683,539
2,NaN,Algeria,28.0339,1.6596,2418
3,NaN,Andorra,42.5063,1.5218,696
4,NaN,Angola,-11.2027,17.8739,19


In [20]:
# file = os.path.join('confirmed.csv')
# combined_df.to_csv(file, index=False, header=True)